In [ ]:
# Importing the Libraries
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam

# Performing Data augmentation on training set
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

training_set = train_datagen.flow_from_directory(
    'train',
    target_size = (256, 256),
    batch_size = 32,
    class_mode = 'binary'
)

# Performing Data augmentation on testing set
test_datagen = ImageDataGenerator(rescale = 1./255)

testing_set = test_datagen.flow_from_directory(
    'test',
    target_size = (256, 256),
    batch_size = 32,
    class_mode = 'binary'
)

# Building the Model and adding layers
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [256, 256, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units = 64,kernel_initializer='he_normal', activation = 'relu'))


# Adding dropout layers to reduce overfitting
cnn.add(tf.keras.layers.Dropout(0.2))

# Adding the output layer
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Making the Optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)

# Compiling the model
cnn.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

# Applying early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Training the model with early stopping callback
cnn.fit(
    x = training_set,
    validation_data = testing_set,
    epochs = 15,
    callbacks = [early_stopping]
)


In [ ]:
# Saving the model for future use
tf.keras.models.save_model(cnn, './')